In [4]:
import pycocotools
from ultralytics import YOLO
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
import yaml
import torch
import shutil
import wandb

In [6]:
train_imgs_dir = "data/training_images"
train_labels = "data/train_solution_bounding_boxes (1).csv"
test_imgs_dir = "data/testing_images"

In [13]:
imgs_list = list(sorted(os.listdir(train_imgs_dir)))
print(imgs_list)
idxs = list(range(len(imgs_list)))
np.random.shuffle(idxs)
train_idx = idxs[:int(0.8*len(idxs))]
val_idx = idxs[int(0.8*len(idxs)):]

['vid_4_1000.jpg', 'vid_4_10000.jpg', 'vid_4_10020.jpg', 'vid_4_10040.jpg', 'vid_4_10060.jpg', 'vid_4_10080.jpg', 'vid_4_10100.jpg', 'vid_4_10120.jpg', 'vid_4_10140.jpg', 'vid_4_10160.jpg', 'vid_4_10180.jpg', 'vid_4_1020.jpg', 'vid_4_10200.jpg', 'vid_4_10220.jpg', 'vid_4_10240.jpg', 'vid_4_10260.jpg', 'vid_4_10280.jpg', 'vid_4_10300.jpg', 'vid_4_10320.jpg', 'vid_4_10340.jpg', 'vid_4_10360.jpg', 'vid_4_10380.jpg', 'vid_4_1040.jpg', 'vid_4_10400.jpg', 'vid_4_10420.jpg', 'vid_4_10440.jpg', 'vid_4_10460.jpg', 'vid_4_10480.jpg', 'vid_4_10500.jpg', 'vid_4_10520.jpg', 'vid_4_10540.jpg', 'vid_4_10560.jpg', 'vid_4_10580.jpg', 'vid_4_1060.jpg', 'vid_4_10600.jpg', 'vid_4_10620.jpg', 'vid_4_10640.jpg', 'vid_4_10660.jpg', 'vid_4_10680.jpg', 'vid_4_10700.jpg', 'vid_4_10720.jpg', 'vid_4_10740.jpg', 'vid_4_10760.jpg', 'vid_4_10780.jpg', 'vid_4_1080.jpg', 'vid_4_10800.jpg', 'vid_4_10820.jpg', 'vid_4_10840.jpg', 'vid_4_10860.jpg', 'vid_4_10880.jpg', 'vid_4_10900.jpg', 'vid_4_10920.jpg', 'vid_4_10940.jpg

In [31]:
!mkdir "working/data"
!mkdir "working/data/images"

!mkdir "working/data/images/train"
!mkdir "working/data/images/val"

########################################
!mkdir "working/data/labels"

!mkdir "working/data/labels/train"
!mkdir "working/data/labels/val"

A subdirectory or file working/data already exists.


In [32]:
root_dir = "working/data"
labels_dir = "working/data/labels"
images_dir = "working/data/images"

In [37]:
df = pd.read_csv(train_labels)
width = 676
height = 380

df['class'] = 0

df.rename(columns = {'image':'img_name'}, inplace=True)
# print(df)

df["x_centre"] = (df["xmin"] + df['xmax'])/2
df["y_centre"] = (df["ymin"] + df["ymax"])/2
df["width"] = (df["xmax"] - df["xmin"])
df["height"] = (df["ymax"] - df["ymin"])

df["x_centre"] = df["x_centre"]/width
df["y_centre"] = df["y_centre"]/height
df["width"] = df["width"]/width
df["height"] = df["height"]/height

df_yolo = df[["img_name", "class", "x_centre", "y_centre", "width", "height"]]
df_yolo.head()
              

,img_name,class,x_centre,y_centre,width,height
0,vid_4_1000.jpg,0,0.450434,0.539817,0.068741,0.095238
1,vid_4_10000.jpg,0,0.100217,0.557191,0.155572,0.129987
2,vid_4_10040.jpg,0,0.444645,0.543678,0.181621,0.157014
3,vid_4_10020.jpg,0,0.833213,0.531451,0.197540,0.155727
4,vid_4_10060.jpg,0,0.110347,0.559122,0.171491,0.136422


In [80]:
for idx, img_name in enumerate(imgs_list):
    subset = "train"
    if idx in val_idx:
        subset = "val"
    # print(img_name[:-4])
    if np.isin(img_name, df_yolo["img_name"]):
        columns = ["class", "x_centre", "y_centre", "width", "height"]
        img_bbox = df_yolo[df_yolo["img_name"] == img_name][columns].values
        # print(type(img_bbox))
        # print(img_bbox)
        
        label_file_path = os.path.join(labels_dir, subset, f"{img_name[:-4]}.txt")
        with open(label_file_path, "w+") as f:
            for row in img_bbox:
                # print(type(row))
                text = " ".join(row.astype(str))
                
                # print(text)
                f.write(text)
                f.write("\n")
        
    old_image_path = os.path.join(train_imgs_dir, img_name)
    new_image_path = os.path.join(images_dir, subset, img_name)
    shutil.copy(old_image_path, new_image_path)
                
        

In [81]:
yolo_format = dict(path = "D:/JJPES/projects/Yolo_car_detection/working/data", train="D:/JJPES/projects/Yolo_car_detection/working/data/images/train", val="D:/JJPES/projects/Yolo_car_detection/working/data/images/val", nc=1, names={0:"car"})
with open("working/yolo.yaml", "w") as outfile:
    yaml.dump(yolo_format, outfile, default_flow_style=False)

In [82]:
my_secret = "6b360d8d41ebffe97e09ee9bfb8890b9d3bcc245"
wandb.login(key = my_secret)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [85]:
model = YOLO('yolov8m.pt')

In [86]:
model.train(data='working/yolo.yaml', epochs=2, patience=5, batch=8, lr0=0.0005, imgsz=640)

New https://pypi.org/project/ultralytics/8.0.166 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.165  Python-3.11.4 torch-2.0.1+cpu CPU (Intel Core(TM) i7-7600U 2.80GHz)
engine\trainer: task=detect, mode=train, model=yolov8m.pt, data=working/yolo.yaml, epochs=2, patience=5, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_

RuntimeError: Dataset 'working/yolo.yaml' error  
Dataset 'working/yolo.yaml' images not found , missing path 'D:\JJPES\projects\Yolo_car_detection\datasets\working\data\working\data\images\val'
Note dataset download directory is 'D:\JJPES\projects\Yolo_car_detection\datasets'. You can update this in 'C:\Users\admin\AppData\Roaming\Ultralytics\settings.yaml'